In [44]:
import pandas as pd
import numpy as np
import time
import datetime
import gc
from sklearn.preprocessing import LabelEncoder,OneHotEncoder


In [45]:
path =  '/Users/apple/Desktop/data/'
   
train = pd.read_csv(path+'train_all.csv')
test = pd.read_csv(path+'test_all.csv')
data = pd.concat([train, test])


In [46]:
# 商品的曝光时间，计算每个商品在每个小时的广告时间里出现的次数
exp_hour = data.groupby(['item_id', 'hour']).instance_id.nunique().to_frame()
exp_hour.columns = ['item_hour_cnt']
exp_hour = exp_hour.reset_index()
exp_hour['item_hour_cnt_ratio'] = round(exp_hour.item_hour_cnt / exp_hour.groupby('item_id').item_hour_cnt.transform(np.sum), 5)
exp_hour = exp_hour[['item_id', 'hour', 'item_hour_cnt_ratio']]
data = data.merge(exp_hour, how='left', on=['item_id', 'hour'])

In [47]:
 # 商品分时间段曝光比例，得到每个item_id在每个时间段实例数量占比
exp_tz = data.groupby(['item_id', 'maphour']).instance_id.nunique().to_frame()
exp_tz.columns = ['item_maphour_cnt']
exp_tz = exp_tz.reset_index()
exp_tz['item_maphour_cnt_ratio'] = round(exp_tz.item_maphour_cnt / exp_tz.groupby('item_id').item_maphour_cnt.transform(np.sum), 5)
exp_tz = exp_tz[['item_id', 'maphour', 'item_maphour_cnt_ratio']]
data = data.merge(exp_tz, how='left', on=['item_id', 'maphour'])

In [48]:
# 用户id每小时曝光数，得到每个user_id在每个hour的实例数量占比
exp_hour = data.groupby(['user_id', 'hour']).instance_id.nunique().to_frame()
exp_hour.columns = ['user_hour_cnt']
exp_hour = exp_hour.reset_index()
exp_hour['user_hour_cnt_ratio'] = round(exp_hour.user_hour_cnt / exp_hour.groupby('user_id').user_hour_cnt.transform(np.sum), 5)
exp_hour = exp_hour[['user_id', 'hour', 'user_hour_cnt_ratio']]
data = data.merge(exp_hour, how='left', on=['user_id', 'hour'])

In [49]:
# 每个user_id在每个时间段的实例数量
exp_tz = data.groupby(['user_id', 'maphour']).instance_id.nunique().to_frame()
exp_tz.columns = ['user_maphour_cnt']
exp_tz = exp_tz.reset_index()
exp_tz['user_maphour_cnt_ratio'] = round(exp_tz.user_maphour_cnt / exp_tz.groupby('user_id').user_maphour_cnt.transform(np.sum), 5)
exp_tz = exp_tz[['user_id', 'maphour', 'user_maphour_cnt_ratio']]
data = data.merge(exp_tz, how='left', on=['user_id', 'maphour'])
gc.collect() #垃圾回收机制

0

In [50]:
#user_id和context_timestamp进行分组，并统计每个分组中的不重复的数量
add = pd.DataFrame(data.groupby(["user_id", "context_timestamp"]).instance_id.nunique()).reset_index()
add.columns = ["user_id", "context_timestamp", "same_time_expo_cnt"]
data = data.merge(add, on=["user_id", "context_timestamp"], how="left")

In [51]:
# 小时均值特征
grouped = data.groupby('user_id')['hour'].mean().reset_index()
grouped.columns = ['user_id', 'user_mean_hour']
data = data.merge(grouped, how='left', on='user_id')
grouped = data.groupby('item_id')['hour'].mean().reset_index()
grouped.columns = ['item_id', 'item_mean_hour']
data = data.merge(grouped, how='left', on='item_id')
grouped = data.groupby('item_brand_id')['hour'].mean().reset_index()
grouped.columns = ['item_brand_id', 'brand_mean_hour']
data = data.merge(grouped, how='left', on='item_brand_id')
grouped = data.groupby('shop_id')['hour'].mean().reset_index()
grouped.columns = ['shop_id', 'shop_mean_hour']
data = data.merge(grouped, how='left', on='shop_id')

In [52]:
# 年龄均值特征
grouped = data.groupby('user_id')['user_age_level'].mean().reset_index()
grouped.columns = ['user_id', 'user_mean_age']
data = data.merge(grouped, how='left', on='user_id')
grouped = data.groupby('item_id')['user_age_level'].mean().reset_index()
grouped.columns = ['item_id', 'item_mean_age']
data = data.merge(grouped, how='left', on='item_id')
grouped = data.groupby('item_brand_id')['user_age_level'].mean().reset_index()
grouped.columns = ['item_brand_id', 'brand_mean_age']
data = data.merge(grouped, how='left', on='item_brand_id')
grouped = data.groupby('shop_id')['user_age_level'].mean().reset_index()
grouped.columns = ['shop_id', 'shop_mean_age']
data = data.merge(grouped, how='left', on='shop_id')

In [53]:
 #小时特征,每个用户计算活动
add = pd.DataFrame(data.groupby(["user_id"]).hour.nunique()).reset_index()
add.columns = ["user_id", "user_active_hour"]
data = data.merge(add, on=["user_id"], how="left")

In [54]:
 # 活跃item_id数特征
add = pd.DataFrame(data.groupby(["item_category_list", "day"]).item_id.nunique()).reset_index()
add.columns = ["item_category_list", "day", "category_day_active_item"]
data = data.merge(add, on=["item_category_list", "day"], how="left")

In [55]:
# 活跃city数特征
add = pd.DataFrame(data.groupby(["user_id", "day"]).item_city_id.nunique()).reset_index()
add.columns = ["user_id", "day", "user_day_active_city"]
data = data.merge(add, on=["user_id", "day"], how="left")

In [56]:
#活跃user数特征
add = pd.DataFrame(data.groupby(["item_id", "day"]).user_id.nunique()).reset_index()
add.columns = ["item_id", "day", "item_day_active_user"]
data = data.merge(add, on=["item_id", "day"], how="left")
add = pd.DataFrame(data.groupby(["shop_id", "day"]).user_id.nunique()).reset_index()
add.columns = ["shop_id", "day", "shop_day_active_user"]
data = data.merge(add, on=["shop_id", "day"], how="left")
add = pd.DataFrame(data.groupby(["item_brand_id", "day"]).user_id.nunique()).reset_index()
add.columns = ["item_brand_id", "day", "brand_day_active_user"]
data = data.merge(add, on=["item_brand_id", "day"], how="left")
add = pd.DataFrame(data.groupby(["item_category_list", "day"]).user_id.nunique()).reset_index()
add.columns = ["item_category_list", "day", "category_day_active_user"]
data = data.merge(add, on=["item_category_list", "day"], how="left")


add = pd.DataFrame(data.groupby(["item_id", "day", "hour"]).user_id.nunique()).reset_index()
add.columns = ["item_id", "day", "hour", "item_hour_active_user"]
data = data.merge(add, on=["item_id", "day", "hour"], how="left")
add = pd.DataFrame(data.groupby(["shop_id", "day", "hour"]).user_id.nunique()).reset_index()
add.columns = ["shop_id", "day", "hour", "shop_hour_active_user"]
data = data.merge(add, on=["shop_id", "day", "hour"], how="left")


In [57]:
# 活跃shop数特征
add = pd.DataFrame(data.groupby(["user_id", "day"]).shop_id.nunique()).reset_index()
add.columns = ["user_id", "day", "user_day_active_shop"]
data = data.merge(add, on=["user_id", "day"], how="left")

# 活跃brand数特征 
add = pd.DataFrame(data.groupby(["user_id", "day"]).item_brand_id.nunique()).reset_index()
add.columns = ["user_id", "day", "user_day_active_brand"]
data = data.merge(add, on=["user_id", "day"], how="left")

add = pd.DataFrame(data.groupby(["user_id", "day", "hour"]).item_brand_id.nunique()).reset_index()
add.columns = ["user_id", "day", "hour", "user_hour_active_brand"]
data = data.merge(add, on=["user_id", "day", "hour"], how="left")

In [58]:
# shop_id仅求均值全局
data_shop_item = data[['shop_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']]
data_shop_item.columns = ['shop_id', 'mean_item_price_level', 'mean_item_sales_level', 'mean_item_collected_level', 'mean_item_pv_level']
shop_item = data_shop_item.groupby(['shop_id']).mean().reset_index()
data = pd.merge(data, shop_item, 'left', on='shop_id')
del data_shop_item
del shop_item

for colname in ['item_category_list', 'item_brand_id', 'item_city_id']:
    gc.collect()
    grouped = data.groupby([colname])
    meancols = ['item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level']
    data_g = grouped[meancols].mean().reset_index()
    colnames = [i for i in data_g.columns]
    for i in range(len(colnames)):
        if colnames[i] != colname:
            colnames[i] += '_mean_by_'+colname.split('_')[1]
    data_g.columns = colnames
    data=pd.merge(data, data_g, how='left', on=colname)
    colnames = colnames[1:]
    for i in range(len(colnames)):
        data[colnames[i]+'_ratio'] = round((data[meancols[i]]/data[colnames[i]]), 5)


In [59]:
 ############挖掘新的特征###########
del data['item_category_list']
gc.collect()

# 得到全部训练集
print('经过处理后,全部训练集最终维度:', data.shape)
data.to_csv(path+'all_01.csv', index=False)

# 得到7号训练集
data = data.loc[data.day==7]
print('7号训练集,最终维度:', data.shape)
print(data.columns.tolist())
data.to_csv(path+'day7_01.csv', index=False)

经过处理后,全部训练集最终维度: (99085, 87)
7号训练集,最终维度: (10184, 87)
['instance_id', 'item_id', 'item_property_list', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id', 'context_timestamp', 'context_page_id', 'predict_category_property', 'shop_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description', 'is_trade', 'time', 'day', 'hour', 'minute', 'maphour', 'mapmin', 'item_category_1', 'item_category_2', 'item_hour_cnt_ratio', 'item_maphour_cnt_ratio', 'user_hour_cnt_ratio', 'user_maphour_cnt_ratio', 'same_time_expo_cnt', 'user_mean_hour', 'item_mean_hour', 'brand_mean_hour', 'shop_mean_hour', 'user_mean_age', 'item_mean_age', 'brand_mean_age', 'shop_mean_age', 'user_active_hour', 'category_day_active_item', 'user_day_active_city', 'item_day_active_use

## eda

In [2]:
import pandas as pd
import matplotlib.pyplot as plt


# 选择要可视化的特征
features_to_visualize = ['user_mean_hour', 'item_mean_age', 'user_day_active_city', 'shop_mean_hour']

# 绘制直方图
plt.figure(figsize=(12, 8))
for feature in features_to_visualize:
    plt.hist(data[feature].dropna(), bins=30, alpha=0.5, label=feature)

plt.title('Distribution of New Features')
plt.xlabel('Feature Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()


NameError: name 'data' is not defined

<Figure size 864x576 with 0 Axes>